In [46]:
import torch
#GPU使用可能確認
print(torch.cuda.is_available())
#使用できるデバイス数確認
print(torch.cuda.device_count())

True
2


In [47]:
!nvidia-smi

Sun Jul 30 13:31:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 54%   51C    P8    42W / 350W |   9784MiB / 24576MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:C1:00.0 Off |                  N/A |
| 42%   

In [48]:
! pip install transformers
! pip install mojimoji
! pip install pyknp

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import random
import re
import string
import time
from tqdm import tqdm
import mojimoji
import pandas as pd
from transformers import BertConfig, BertModel, BertTokenizer
from pyknp import Juman

###　ターミナルで形態素解析juman++のインストール
https://qiita.com/Gushi_maru/items/ee434b5bc9f020c8feb6
・jumanpp-2.0.0-rc3 download
wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz
・unzip a file
tar xvf jumanpp-2.0.0-rc3.tar.xz
・コンパイラのインストール
sudo apt update -y
sudo apt upgrade -y
sudo apt install build-essential -y
・cmakeインストール
sudo apt install cmake -y
・コンパイラ用フォルダ作成
cd [jumanディレクトリのパスの指定]
mkdir build
・juman++コンパイル
cd [buildのパス指定]
cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
make
・コンパイル完了したらインストール開始
sudo make install


In [50]:
#juman++動作確認
! echo すももももももももももも | jumanpp

すもも すもも すもも 名詞 6 普通名詞 1 * 0 * 0 "自動獲得:テキスト"
@ すもも すもも すもも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:酸桃/すもも 自動獲得:EN_Wiktionary"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
EOS


### jiuman++のバージョン確認

In [51]:
! jumanpp -v

Juman++ Version: 2.0.0-rc3 / Dictionary: 20190731-356e143 / LM: K:20190430-7d143fb L:20181122-b409be68 F:20171214-9d125cb


In [52]:
# JUMANの動作確認
from pyknp import Juman
text = "自然言語処理のBERTについて学習中です。"
juman = Juman()
result =juman.analysis(text)
result = [mrph.midasi for mrph in result.mrph_list()]
print(text)
print(result)

自然言語処理のBERTについて学習中です。
['自然', '言語', '処理', 'の', 'BERT', 'に', 'ついて', '学習', '中', 'です', '。']


### ダウンロードしたモデルのロード

In [53]:
import torch
vocab_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Japanese_L-12_H-768_A-12_E-30_BPE 2/vocab.txt"
config_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Japanese_L-12_H-768_A-12_E-30_BPE 2/bert_config.json"
model_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Japanese_L-12_H-768_A-12_E-30_BPE 2/pytorch_model.bin"

In [54]:
bert_tokenizer = BertTokenizer(vocab_file, do_lower_case=False, do_basic_tokenize=False)

In [55]:
from pyknp import Juman

class JumanTokenizer():
    def __init__(self):
        self.juman = Juman()

    def tokenize(self, text):
        result = self.juman.analysis(text)
        return [mrph.midasi for mrph in result.mrph_list()]

In [56]:
juman_tokenizer = JumanTokenizer()

In [57]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
    
def preprocessing_text(text):
       
        text = mojimoji.han_to_zen(text) 
        
        text = re.sub(r'[0-9 ０-９]+', '0', text)  

        text = re.sub('\r', '', text)
        text = re.sub('\n', '', text)

        text = text.replace("、", " ")
        text = text.replace("。", " ")
        
        for p in string.punctuation:
            if (p == ".") or (p == ","):
                continue
            else:
                text = text.replace(p, " ")
            return text

def tokenizer_with_preprocessing(text):
        text = preprocessing_text(text)
        tokens = juman_tokenizer.tokenize(text)
        bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
        ids = bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens+ ["[SEP]"])
        return ids

def preprocessing_sentences(text):
        for i in range(len(text)):
            text[i] = preprocessing_text(text[i])
        return text

### 訓練データ

In [132]:
# 作成したデータの読み込み
df_train = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/train_data(5).csv", header=0)
#df_train = df_train.dropna(how='any') # nanのところは落とす（欠損値の削除）

In [133]:
# データの確認
print(f'データサイズ： {df_train.shape}')
display(df_train.sample(10))

データサイズ： (7000, 2)


,text,label
5210,ユニバーサルスタジオハリウッド の スタジオツアーノーカット 動画 って の 観 て たら ...,1
1781,もう 5年 も 一緒 に いる ん だ よ 、 、 次 は Disney か な ??? #...,0
674,ハウステンボス 光 の 王国 大 迫力 の 綺麗 すぎる くらい の イルミ だっ た ずっ...,1
157,ユニバ３回ぐらいしか行ったことない私ですら迷わんかったぞ,0
5845,ディズニー、クリスマスにも行こうよって。どこまで本気なのかわからないけど実現したらいいな。,0
2265,休日のパワーチャージは、コンラッド東京の限定ブランチで。\n＜ＵＲＬ＞,0
2870,俺だって恋人とディズニー行ってカチューシャ付けたりしたい！！！,0
6838,ＴＤＲ？？？東京ディズニーランド？？？\n（´・ε・｀）？？？\nがっつりＵＳＪ派だわ\n\...,0
4806,超特急 ユニバーサル・スタジオ・ジャパン ライブ チケット 譲 : 12 / 25 スタンデ...,0
1574,a ー nation 2018 【 求 】 8 / 4 長崎 ハウステンボス 2 連 A 2...,1


### テストデータ

In [134]:
# 作成したデータの読み込み
df_test = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/test_data(5).csv",header=0)

In [135]:
print(f'データサイズ： {df_test.shape}')
display(df_test.sample(10))

データサイズ： (1998, 2)


,text,label
1495,昨日 の 夜 mommy に 日曜 ディズニー の パレード ??? って 目 輝か せ て...,0
650,この 前 根岸 さん に 、 ねぇ ! ディズニー 行か ない の !? ディズニー !! ...,1
196,ユニバいきたい\n\n早く\n年パスを買わなきゃ,0
1507,来月 は ディズニー ヾ (@⌒ ー ⌒@) ノ ?,0
1436,ユニバ の ハリポタワールド の グッズ 調べ た けど なかなか なお 値段 する みたい...,1
184,プリクラとってる時にディズニーのチケットのサプライズ死ぬまでにされてみたい,0
1010,マツコの知らない世界 の ディズニーシー 特集 すごい おもしろかっ た 。 なぜ そう な...,1
31,昨日友達とオルゴールミュージアム・ピクサー展行った！！\nディズニーメドレーがちよかったわ\...,1
242,【レゴランド】来場者低迷、隣接店舗が２か月で閉店売り上げが想定の１０分の１,0
919,マカティシャングリラマニラ ( Makati Shangri ー La ) に 宿泊 し ま...,0


### valデータ

In [136]:
# 作成したデータの読み込み
df_val = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/val_data(5).csv",header=0)

In [137]:
print(f'データサイズ： {df_val.shape}')
display(df_val.sample(10))

データサイズ： (996, 2)


,text,label
432,【 生放送 】 USJ 及び 大阪 を 勉強 する 枠 を 開始 し まし た 。 < UR...,0
546,ディズニーランド クリスマス 関連 ブログ 更新 中 ! よろしく です < URL >,0
907,2018-10-06t21:21:21z新潟地方気象台【天気概況】佐渡では、7日朝まで暴風に...,0
375,次 ユニバ 行く 時 、 レイブンクロー か スリザリン の ネクタイ 買う ねん はあ 、...,1
50,バンドアレンジしてみましたー（･∀･）！！よかったら聴いてみてね！！【全１０曲メドレー】ディ...,1
460,6月 の 大阪 楽しみ すぎ て … コンラッド 予約 取れ た ＼ (^o^)/ 5月 は...,1
259,Ｄｉｓｎｅｙ行った時に撮った風景のやつ\n\nつぶやき今日始めたばっかでフォロワー少ないから...,1
746,え 、 この ディズニー の バッグ 見 た こと ない と 思っ て 確認 し たら 香港...,0
388,6 . 3 な あや と ユニバ 行っ て き た ! 暑 すぎ ほんま に 、 絶対 焼け...,1
835,2日目 は モーニング バイキング し て 日光江戸村 へ 。 せっかく だし と 女 剣士...,0


In [138]:
# モデルに飲ませるデータと、ラベルを準備
text_train = df_train.text.values
labels_train = df_train.label.values


## 確認
print(' Original: ', text_train[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_train[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_train[0]))

 Original:  え 、 ピューロランド 行き たい 。 。 。 誰か 連れ て っ て ← 絶対 SKY ー HI 可愛い やん 。 いや 、 カッコイイ パターン も あり ?
Tokenized:  ['え', '\u3000', '\u3000', '\u3000', 'ピュー', 'ロランド', '\u3000', '行き', '\u3000', 'たい', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '誰', 'か', '\u3000', '連れ', '\u3000', 'て', '\u3000っ', '\u3000', 'て', '\u3000', '←', '\u3000', '絶対', '\u3000', 'ＳＫＹ', '\u3000', 'ー', '\u3000', 'ＨＩ', '\u3000', '可愛い', '\u3000', 'やん', '\u3000', '\u3000', '\u3000', 'いや', '\u3000', '\u3000', '\u3000', 'カッコイイ', '\u3000', 'パターン', '\u3000', 'も', '\u3000', 'あり', '\u3000', '？']
Token IDs:  [2, 2639, 1, 1, 1, 16249, 1, 1, 2003, 1, 755, 1, 1, 1, 1, 1, 1, 1, 2437, 90, 1, 6550, 1, 3402, 1, 1, 3402, 1, 1, 1, 4073, 1, 1, 1, 20338, 1, 24204, 1, 29850, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4710, 1, 23, 1, 125, 1, 1566, 3]


In [139]:
# モデルに飲ませるデータと、ラベルを準備
text_val = df_val.text.values
labels_val = df_val.label.values

## 確認
print(' Original: ', text_val[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_val[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_val[0]))

 Original:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですね
なるほどこの世界はなろう小説でしたか
Tokenized:  ['そう', 'いえば', '大阪', 'の', 'ウンタラ', 'で', 'は', 'ＵＳＪ', 'の', 'ウンタラ', 'を', 'ウンタラ', 'する', 'そうです', 'ね', 'なるほど', 'この', '世界', 'は', 'なろう', '小説', 'でした', 'か']
Token IDs:  [2, 2208, 10922, 340, 5, 1, 13, 9, 1, 5, 1, 10, 1, 22, 1, 2382, 1, 46, 118, 9, 17070, 728, 14947, 90, 3]


In [140]:
# モデルに飲ませるデータと、ラベルを準備
text_test = df_test.text.values
labels_test = df_test.label.values

## 確認
print(' Original: ', text_test[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_test[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_test[0]))

 Original:  レゴランドの向いにホテルが立ってるんですよー
私も今日、初めて行きました；；
レストランはビュッフェタイプでした。
他にもバーとかありましたよ？
少々お高いですが、わりと楽しめました！
Tokenized:  ['レゴ', 'ランド', 'の', '向い', 'に', 'ホテル', 'が', '立って', 'る', 'んです', 'よー', '私', 'も', '今日', '\u3000', '初めて', '行き', 'ました', '；', '；', 'レストラン', 'は', 'ビュッフェ', 'タイプ', 'でした', '\u3000', '他', 'に', 'も', 'バー', 'と', 'か', 'あり', 'ました', 'よ', '？', '少々', 'お', '高い', 'です', 'が', '\u3000', 'わりと', '楽しめ', 'ました', '！']
Token IDs:  [2, 1, 5802, 5, 1, 8, 1762, 11, 7234, 3282, 10307, 1, 1038, 23, 2281, 1, 526, 2003, 4561, 4123, 4123, 4825, 9, 1, 1483, 14947, 1, 151, 8, 23, 1304, 12, 90, 125, 4561, 1291, 1566, 23151, 273, 537, 3338, 11, 1, 1, 1, 4561, 228, 3]


In [141]:
text_train = preprocessing_sentences(text_train)
text_val = preprocessing_sentences(text_val)
text_test = preprocessing_sentences(text_test)
print('Preprocessing: ', text_train[0])
print('Preprocessing: ', text_val[0])
print('Preprocessing: ', text_test[0])

Preprocessing:  え　 　ピューロランド　行き　たい　 　 　 　誰か　連れ　て　っ　て　←　絶対　ＳＫＹ　ー　ＨＩ　可愛い　やん　 　いや　 　カッコイイ　パターン　も　あり　？
Preprocessing:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですねなるほどこの世界はなろう小説でしたか
Preprocessing:  レゴランドの向いにホテルが立ってるんですよー私も今日 初めて行きました；；レストランはビュッフェタイプでした 他にもバーとかありましたよ？少々お高いですが わりと楽しめました！


### IDへ変換

In [142]:
input_ids_train = []
attention_masks_train = []
max_len = 128

# 1文づつ処理
for sentence in text_train:
    encoded_dict_train = bert_tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_train.append(encoded_dict_train['input_ids'])
    attention_masks_train.append(encoded_dict_train['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)

# tensor型に変換
labels_train = torch.tensor(labels_train)

# 確認
print('Original: ', text_train[0])
print('Token IDs:', input_ids_train[0])

/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  え　 　ピューロランド　行き　たい　 　 　 　誰か　連れ　て　っ　て　←　絶対　ＳＫＹ　ー　ＨＩ　可愛い　やん　 　いや　 　カッコイイ　パターン　も　あり　？
Token IDs: tensor([    2,  2639, 16249,   598,  2401,  2003,   755,  2437,   856,  6550,
         3402, 14443,  3402,     1,  4073, 14229,  1192, 20338, 24204, 29850,
           34,  2029,   142,  2143, 22565,   643,   643,  4710,    23,   125,
         1566,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   

In [143]:
input_ids_val = []
attention_masks_val = []
max_len = 128


# 1文づつ処理
for sentence in text_val:
    encoded_dict_val = bert_tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_val.append(encoded_dict_val['input_ids'])
    attention_masks_val.append(encoded_dict_val['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val = torch.cat(attention_masks_val, dim=0)

# tensor型に変換
labels_val = torch.tensor(labels_val)

# 確認
print('Original: ', text_val[0])
print('Token IDs:', input_ids_val[0])

Original:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですねなるほどこの世界はなろう小説でしたか
Token IDs: tensor([    2,  2208,   574,  9830,  1448, 11136,   422,  7070, 22644,   429,
         7019,  5901,  2007,   422,  7070, 22644, 25246,  7070, 22644,  3244,
         3581, 12323,  3861, 10124, 17108,  1126,   422,  3390,  4971,  7019,
          275,  3958,  5574,  5748,   429,  1033,   856,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,    

In [144]:
input_ids_test = []
attention_masks_test = []
max_len = 128

# 1文づつ処理
for sentence in text_test:
    encoded_dict_test = bert_tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_test.append(encoded_dict_test['input_ids'])
    attention_masks_test.append(encoded_dict_test['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)

# tensor型に変換
labels_test = torch.tensor(labels_test)

# 確認
print('Original: ', text_test[0])
print('Token IDs:', input_ids_test[0])

Original:  レゴランドの向いにホテルが立ってるんですよー私も今日 初めて行きました；；レストランはビュッフェタイプでした 他にもバーとかありましたよ？少々お高いですが わりと楽しめました！
Token IDs: tensor([    2,     1,     1,     1, 29823,  1878,  3503, 26733, 28656, 11716,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

### データローダーの準備

In [145]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train) # input_ids_train, attention_masks_train, labels_train を使ってデータセットを作成
val_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# データローダーの作成
batch_size = 16

# datsets = [データセット全て]
# Dataloader = [[batch_1], [batch_2], ... [batch_n]]

# 訓練データローダー
train_dataloader = DataLoader( # Dataloader : データセットからデータをバッチサイズに固めて返すモジュール
    train_dataset,  
    sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化（sampler : datasetsのバッチの固め方を決める事のできる設定）
    batch_size = batch_size
)

# 検証データローダー
validation_dataloader = DataLoader(
    val_dataset, 
    sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
    batch_size = batch_size
)

# テストデータローダー
test_dataloader = DataLoader(
    test_dataset, 
    sampler = SequentialSampler(test_dataset), # 順番にデータを取得してバッチ化
    batch_size = batch_size
)

In [146]:
config = BertConfig.from_json_file(config_file)
model = BertModel.from_pretrained(model_file,config=config)

Some weights of the model checkpoint at /home/sakulab/workspace/yaguchi/research/Master/code/LLM/Japanese_L-12_H-768_A-12_E-30_BPE 2/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [147]:
class BertClassifier(nn.Module):
  def __init__(self):
    super(BertClassifier, self).__init__()
    self.bert = model
    
    # BERTの隠れ層の次元数は768, カテゴリ数が2
    self.linear = nn.Linear(768, 2)
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)

  def forward(self, input_ids):
    # last_hidden_stateを受け取る
    vec = self.bert(input_ids)
    # 先頭トークンclsのベクトルだけ取得
    vec = vec[0][:,0,:]   #変更
    vec = vec.view(-1, 768)
    # 全結合層でクラス分類用に次元を変換
    out = self.linear(vec)
    
    return out

model = BertClassifier()

### ファインチューニング

In [148]:
#ファインチューニング
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

for name, param in model.linear.named_parameters():
    param.requires_grad = True

# 最適化手法の設定
"""
optimizer = Adam([
    {'params': model.bert.encoder.layer[-1].parameters(), 'lr': 2e-5},
    {'params': model.linear.parameters(), 'lr': 2e-5}
], betas=(0.9, 0.999))
"""
#最適化関数の設定
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [149]:
EarlyStopping_path = '/home/sakulab/workspace/yaguchi/research/Master/code/LLM/BERT/京大bert/checkpoint/val5(epoch10)_bert_checkpoint_model.pth'

class EarlyStopping:
    """earlystoppingクラス"""

    def __init__(self, patience=5, verbose=False, path= EarlyStopping_path ):
        """引数：最小値の非更新数カウンタ、表示設定、モデル格納path"""

        self.patience = patience    #設定ストップカウンタ
        self.verbose = verbose      #表示の有無
        self.counter = 0            #現在のカウンタ値
        self.best_score = None      #ベストスコア
        self.early_stop = False     #ストップフラグ
        self.val_loss_min = np.Inf   #前回のベストスコア記憶用
        self.path = path             #ベストモデル格納path

    def __call__(self, val_loss, model):
        """
        特殊(call)メソッド
        実際に学習ループ内で最小lossを更新したか否かを計算させる部分
        """
        score = -val_loss

        if self.best_score is None:  #1Epoch目の処理
            self.best_score = score   #1Epoch目はそのままベストスコアとして記録する
            self.checkpoint(val_loss, model)  #記録後にモデルを保存してスコア表示する
        elif score < self.best_score:  # ベストスコアを更新できなかった場合
            self.counter += 1   #ストップカウンタを+1
            if self.verbose:  #表示を有効にした場合は経過を表示
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')  #現在のカウンタを表示する 
            if self.counter >= self.patience:  #設定カウントを上回ったらストップフラグをTrueに変更
                self.early_stop = True
        else:  #ベストスコアを更新した場合
            self.best_score = score  #ベストスコアを上書き
            self.checkpoint(val_loss, model)  #モデルを保存してスコア表示
            self.counter = 0  #ストップカウンタリセット

    def checkpoint(self, val_loss, model):
        '''ベストスコア更新時に実行されるチェックポイント関数'''
        if self.verbose:  #表示を有効にした場合は、前回のベストスコアからどれだけ更新したか？を表示
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  #ベストモデルを指定したpathに保存
        self.val_loss_min = val_loss  #その時のlossを記録する

In [150]:
# 学習結果の保存用
history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc':[]
    }

def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    #★EarlyStoppingクラスのインスタンス化とエアearylistoppingの設定
    earlystopping = EarlyStopping(patience=15)

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    

    # ネットワークをGPUへ
    model.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに
               
            else:
                model.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                #inputs = batch.Text[0].to(device)  # 文章 input_ids
                inputs = batch[0].to(device)  # 文章 input_ids
                #labels = batch.Label.to(device)  # ラベル
                labels = batch[2].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # Bertに入力
                    outputs= model(inputs)
                    
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                       
                      
                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)
      
            # epochごとのlossと正解率
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

            
            if phase == 'train':
                 history['train_loss'].append(epoch_loss)
                 history['train_acc'].append(epoch_acc)
            else:
                history['val_loss'].append(epoch_loss)   
                history['val_acc'].append(epoch_acc)
                
        earlystopping(epoch_loss, model) #callメソッド呼び出し
        if earlystopping.early_stop: #ストップフラグがTrueの場合、breakでforループを抜ける
              print("Early Stopping!")
              break

    return model

In [151]:
num_epochs = 10
dataloaders_dict = {"train": train_dataloader, "val": validation_dataloader}
model_trained = train_model(model, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
Epoch 1/10 | train |  Loss: 0.5377 Acc: 0.7523
Epoch 1/10 |  val  |  Loss: 0.5156 Acc: 0.7450
Epoch 2/10 | train |  Loss: 0.4892 Acc: 0.7777
Epoch 2/10 |  val  |  Loss: 0.5026 Acc: 0.7681
Epoch 3/10 | train |  Loss: 0.4661 Acc: 0.7916
Epoch 3/10 |  val  |  Loss: 0.4616 Acc: 0.7932
Epoch 4/10 | train |  Loss: 0.4458 Acc: 0.8026
Epoch 4/10 |  val  |  Loss: 0.4762 Acc: 0.7841
Epoch 5/10 | train |  Loss: 0.4357 Acc: 0.8110
Epoch 5/10 |  val  |  Loss: 0.4611 Acc: 0.7892
Epoch 6/10 | train |  Loss: 0.4187 Acc: 0.8156
Epoch 6/10 |  val  |  Loss: 0.4536 Acc: 0.7962
Epoch 7/10 | train |  Loss: 0.4069 Acc: 0.8241
Epoch 7/10 |  val  |  Loss: 0.4565 Acc: 0.7811
Epoch 8/10 | train |  Loss: 0.3968 Acc: 0.8296
Epoch 8/10 |  val  |  Loss: 0.4517 Acc: 0.7882
Epoch 9/10 | train |  Loss: 0.3809 Acc: 0.8403
Epoch 9/10 |  val  |  Loss: 0.4768 Acc: 0.7801
Epoch 10/10 | train |  Loss: 0.3681 Acc: 0.8430
Epoch 10/10 |  val  |  Loss: 0.4737 Acc: 0.7952


In [152]:
# ベストモデルを適応させる
model_trained.load_state_dict(torch.load(EarlyStopping_path))

<All keys matched successfully>

### 正解率の算出

In [153]:
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_trained.eval()   # モデルを検証モードに
model_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

y_pred = []

for batch in tqdm(test_dataloader):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #inputs = batch.Text[0].to(device)  # 文章
    inputs = batch[0].to(device)  # 文章
    #labels = batch.Label.to(device)  # ラベル
    labels = batch[2].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

       
        outputs = model_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        if y_pred == []:
          y_pred = preds
        else:
          y_pred = torch.cat([y_pred, preds], dim=0)
        
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dataloader.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dataloader.dataset), epoch_acc))

100%|██████████| 125/125 [00:03<00:00, 33.26it/s]

テストデータ1998個での正解率：0.8108


In [154]:
y_pred = y_pred.to('cpu')

### 分類評価指標4つでの結果

In [155]:
# 評価指標の算出
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# 評価指標算出メソッドの作成
def print_metrics_by_cutoff(y, y_pred, model, cutoff):
    """
    指定した閾値に基づくAccuracy, Precision, Recall, F1-scoreを出力する
    """
    #y_pred = model.predict_proba(X)[:, 1] >= cutoff
    print('accuracy = {:.4f}'.format(accuracy_score(y_true=y, y_pred=y_pred)))
    print('precision = {:.4f}'.format(precision_score(y_true=y, y_pred=y_pred)))
    print('recall = {:.4f}'.format(recall_score(y_true=y, y_pred=y_pred)))
    print('f1 score = {:.4f}'.format(f1_score(y_true=y, y_pred=y_pred)))
    # Accuracy, Precision, Recall, F1-score
print_metrics_by_cutoff(y=labels_test, y_pred=y_pred, model=model_trained, cutoff=0.5)

accuracy = 0.8108
precision = 0.6533
recall = 0.4701
f1 score = 0.5468


In [1]:
#GPUのメモリ解放
! kill -9 $(lsof -t /dev/nvidia*)

In [2]:
!nvidia-smi

Sun Jul 30 14:00:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 72%   62C    P0   133W / 350W |    322MiB / 24576MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:C1:00.0 Off |                  N/A |
| 42%   